# Pvsamv1Tools Examples

These examples demonstrate how to use PySAM's `Pvsamv1Tools` to size a system and access module and inverter parameters from the System Advisor Model (SAM) libraries, and how to use `ResourceTools` to download a weather file from the NREL National Solar Radiation Databse (NSRDB) and use it for a `Pvsamv1` simulation.

In [ ]:
import PySAM
import PySAM.Pvsamv1 as pv
import PySAM.Pvsamv1Tools as pvtools
import PySAM.ResourceTools as resourcetools
print(PySAM.__version__)

## Use ResourceTools to download a weather file: FetchResourceFiles()

This function downloads weather files from the NREL National Solar Radiation Database (NSRDB). In this example, we are downloading a file for a single location and storing it in the current folder to use in the next example.

Register on the NREL Developer Network to get an NREL API key: https://developer.nrel.gov/signup/ and replace `DEMO_KEY` and `test_email.email.org` with your key and email address. `DEMO_KEY` only allows limited access to the API for testing.

In [ ]:
nsrdb_get = resourcetools.FetchResourceFiles( tech='solar', nrel_api_key='DEMO_KEY', nrel_api_email='test_email@email.org', resource_dir='.' )
lat_lon = (-105.1,39.7)
nsrdb_get.fetch([lat_lon]) # this can be a list of lat_lon to download files for multiple locations
wf_path = nsrdb_get.resource_file_paths_dict[lat_lon]
print(wf_path)

## Run a simulation for the Default Detailed PV - Single Owner configuration

Run a simulation for the defaults using the weather file we just downloaded to get a baseline annual output to compare with the next set of examples.

See https://nrel-pysam.readthedocs.io/en/main/sam-configurations.html#detailed-pv-model-single-owner for a description of the configuration. You can use the approach in this example for other PV and PV-Battery configurations.

In [ ]:
model = pv.default('FlatPlatePVSingleOwner')
model.SolarResource.solar_resource_file = wf_path
model.value('analysis_period', 1)
model.execute()
print(model.Outputs.annual_energy)

## Use Pvsamv1Tools to size the system: size_electrical_parameters()

This function sizes the PV system given a target system capacity in kW, target DC/AC ratio with optional maximum inverter DC voltage, inverter number of inputs (up to four), and maximum number of combiner boxes. (Note that SAM's PV sizing code on the System Design page does not account for the number of combiner boxes.)

For this example, we are sizing a 130,000 kWdc system with a DC/AC ratio of 1.2.

In [ ]:
n_strings, n_combiners, n_inverters, calculated_system_capacity = pvtools.size_electrical_parameters(model, 130000, 1.2)
print(calculated_system_capacity)
print(n_strings)
print(n_inverters)
print(n_combiners)
print(model.value('inverter_count'))
model.execute()
print(model.Outputs.annual_energy)

## Choose a module from SAM's CEC Module library: set_cec_module_library_selection()

This function downloads the latest version of the CSV file that stores SAM's CEC module library parameters from the SAM GitHub repository, and then sets the parameters for the Pvsamv1 `CECPerformanceModelWithModuleDatabase` group to values for the appropriate module from the library.

This requires that you look up the module name in the "Name" column of the CSV file for the module you want to select. In this example, we are downloading parameters for a module with the name "AIMS POWER PV555MONO".

https://github.com/NREL/SAM/blob/patch/deploy/libraries/CEC%20Modules.csv

In [ ]:
model.value('module_model',1)
mod_dict = pvtools.set_cec_module_library_selection(model, "AIMS Power PV555MONO")
print(mod_dict)
print(model.value("cec_v_mp_ref"))

## Choose an inverter from SAM's CEC Inverter library: set_cec_inverter_library_selection()

This function downloads the latest version of the CSV file that stores SAM's CEC inverter library parameters from the SAM GitHub repository, and then sets the parameters for the Pvsamv1 `InverterCECDatabase` group to values for the appropriate inverter from the library.

This requires that you look up the inverter name in the "Name" column of the CSV file for the inverter you want to select. In this example, we are downloading parameters for an inverter with the name "NEXTRACKER LLC: NXPCS3340K-HUD [800V]".

https://github.com/NREL/SAM/blob/patch/deploy/libraries/CEC%2Inverers.csv

In [ ]:
inv_dict = pvtools.set_cec_inverter_library_selection(model, "NEXTRACKER LLC: NXPCS3340K-HUD [800V]")
print(inv_dict)
print(model.value("inv_snl_paco"))

## Run Pvsamv1 with new module and inverter

In [ ]:
model.value('inverter_model', 0)
n_strings, n_combiners, n_inverters, calculated_system_capacity = pvtools.size_electrical_parameters(model, 130000, 1.2)
print(calculated_system_capacity)
print(n_strings)
print(n_inverters)
print(model.value('inverter_count'))
print(model.value('cec_v_mp_ref'))
print(model.value('cec_i_mp_ref'))
print(model.value('subarray1_modules_per_string'))
model.execute()
print(model.Outputs.annual_energy)